In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from MyWorking.Pytorch.LogisticRegression import x_train, y_train, criterion, y_pred

In [52]:
df = pd.read_csv("train.csv")
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [53]:
df = df.drop("PassengerId", axis=1)
df = df.drop("Name", axis=1)
df = df.drop("Ticket", axis=1)
df = df.drop("Cabin", axis=1)

In [54]:
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S


In [55]:
df["Embarked"].unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [60]:
df.dropna(subset = ['Embarked'], inplace=True)
df['Embarked'].isnull().sum()

np.int64(0)

In [61]:
df['Age'] = df['Age'].fillna(df['Age'].mean())
df['Age'].isnull().sum()

np.int64(0)

In [62]:
Sex_Dict = {"female": 0, "male": 1}
Embarked_Dict = {"C": 0, "S": 1, "Q": 2}

df["Sex"] = df["Sex"].map(Sex_Dict)
df["Embarked"] = df["Embarked"].map(Embarked_Dict)
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,1
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,1


In [63]:
from sklearn.model_selection import train_test_split

y = df["Survived"]
x = df.drop("Survived", axis=1)

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=15,stratify=y)

In [64]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [65]:
import torch
x_train = torch.tensor(x_train,dtype=torch.float32)
x_test = torch.tensor(x_test,dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [90]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TitanicModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(7,16)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(16,8)
        self.out = nn.Linear(8,1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        output = self.out(x)
        return output

model = TitanicModel()


In [91]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [92]:
epochs = 200
for epoch in range(epochs):
    y_pred = model(x_train)

    loss = criterion(y_pred, y_train)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch {epoch+1}: Loss {loss.item():.4f}')


Epoch 10: Loss 0.6093
Epoch 20: Loss 0.5159
Epoch 30: Loss 0.4705
Epoch 40: Loss 0.4606
Epoch 50: Loss 0.4400
Epoch 60: Loss 0.4461
Epoch 70: Loss 0.4269
Epoch 80: Loss 0.4204
Epoch 90: Loss 0.4268
Epoch 100: Loss 0.3995
Epoch 110: Loss 0.4035
Epoch 120: Loss 0.4054
Epoch 130: Loss 0.3979
Epoch 140: Loss 0.3985
Epoch 150: Loss 0.4137
Epoch 160: Loss 0.4008
Epoch 170: Loss 0.4093
Epoch 180: Loss 0.3968
Epoch 190: Loss 0.3969
Epoch 200: Loss 0.3950


In [93]:
model.eval()
with torch.no_grad():
    raw_predictions = model(x_test)
    predictions = (torch.sigmoid(raw_predictions) > 0.5).float()

    accuracy = (predictions == y_test).sum() / len(y_test)
    print(f"Modelin Doğruluk Payı (Accuracy): %{accuracy.item()*100:.2f}")

Modelin Doğruluk Payı (Accuracy): %83.90


In [94]:
x.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,1
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,1


In [98]:
yolcu = [1, 0, 22, 0, 1, 150, 0]

yolcu_scaled = scaler.transform([yolcu])

yolcu_tensor = torch.tensor(yolcu_scaled, dtype=torch.float32)

model.eval()
with torch.no_grad():
    y_prediction = model(yolcu_tensor)
    y_probability = torch.sigmoid(y_prediction)

print(f"Hayatta kalma ihtimali: %{y_probability.item()*100:.2f}")

Hayatta kalma ihtimali: %89.37


/opt/homebrew/Caskroom/miniforge/base/envs/ai_env/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [102]:
test_df = pd.read_csv('test.csv')
passenger_ids = test_df['PassengerId']

test_df['Age'] = test_df['Age'].fillna(test_df['Age'].mean())
test_df['Fare'] = test_df['Fare'].fillna(test_df['Fare'].mean())
test_df['Sex'] = test_df['Sex'].map({"female": 0, "male": 1})
test_df['Embarked'] = test_df['Embarked'].map({"C": 0, "S": 1, "Q": 2})

test_X = test_df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
test_X_scaled = scaler.transform(test_X)
test_tensor = torch.tensor(test_X_scaled, dtype=torch.float32)

model.eval()
with torch.no_grad():
    test_outputs = model(test_tensor)
    predictions = (torch.sigmoid(test_outputs) > 0.5).int().squeeze()

submission = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Survived": predictions.numpy()
})

submission.to_csv('submission.csv', index=False)

In [103]:
# KATDETME
import torch
import pickle

torch.save(model.state_dict(), 'titanic_model.pth')

preprocessing_bundle = {
    "scaler": scaler,
    "Sex_Dict": {"female": 0, "male": 1},
    "Embarked_Dict": {"C": 0, "S": 1, "Q": 2}
}

with open('titanic_prep.pkl', 'wb') as f:
    pickle.dump(preprocessing_bundle, f)

In [104]:
# KAYDEDİLEN MODELİ KULLANMA
import torch
import pickle
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F

class TitanicModel(nn.Module):
    def __init__(self, input_features):
        super(TitanicModel, self).__init__()
        self.fc1 = nn.Linear(input_features, 16)
        self.dropout = nn.Dropout(0.2)
        self.fc2 = nn.Linear(16, 8)
        self.out = nn.Linear(8, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        return self.out(x)

with open('titanic_prep.pkl', 'rb') as f:
    bundle = pickle.load(f)
    scaler = bundle["scaler"]
    Sex_Dict = bundle["Sex_Dict"]
    Embarked_Dict = bundle["Embarked_Dict"]

model = TitanicModel(7)
model.load_state_dict(torch.load('titanic_model.pth'))
model.eval()

yeni_yolcu = [[1, Sex_Dict["female"], 22, 0, 1, 150, Embarked_Dict["C"]]]

yolcu_df = pd.DataFrame(yeni_yolcu, columns=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'])
yolcu_scaled = scaler.transform(yolcu_df)
yolcu_tensor = torch.tensor(yolcu_scaled, dtype=torch.float32)

with torch.no_grad():
    cikti = model(yolcu_tensor)
    olasilik = torch.sigmoid(cikti).item()

print(f"--- ANALİZ TAMAMLANDI ---")
print(f"Hayatta Kalma Şansı: %{olasilik*100:.2f}")
print(f"Durum: {'YAŞAR' if olasilik > 0.5 else 'ÖLDÜ'}")

--- ANALİZ TAMAMLANDI ---
Hayatta Kalma Şansı: %89.37
Durum: YAŞAR


/var/folders/gb/89d1nntx0s50rqj6z6274h400000gn/T/ipykernel_1133/2043824573.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('titanic_mod